In [1]:
# import libraries

import pandas as pd                     # for data manipulation and cleaning
import seaborn as sns                   # for data visulization
import plotly.express as px             # for data visulization
import matplotlib.pyplot as plt         # for data visulization

from sklearn.preprocessing import LabelEncoder

ModuleNotFoundError: No module named 'pandas'

In [2]:
pip install pandas

     ---------------------------------------- 0.0/61.2 kB ? eta -:--:--
     -------------------- ------------------- 30.7/61.2 kB 1.3 MB/s eta 0:00:01
     -------------------- ------------------- 30.7/61.2 kB 1.3 MB/s eta 0:00:01
     ------------------------------- ------ 51.2/61.2 kB 290.5 kB/s eta 0:00:01
     -------------------------------------- 61.2/61.2 kB 296.3 kB/s eta 0:00:00
  Using cached pytz-2023.3.post1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2023.3-py2.py3-none-any.whl (341 kB)
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.6 MB 48

   -------------------- ------------------- 8.0/15.8 MB 86.1 kB/s eta 0:01:31
   -------------------- ------------------- 8.0/15.8 MB 86.1 kB/s eta 0:01:31
   -------------------- ------------------- 8.0/15.8 MB 86.1 kB/s eta 0:01:31
   -------------------- ------------------- 8.0/15.8 MB 86.1 kB/s eta 0:01:31
   -------------------- ------------------- 8.0/15.8 MB 87.1 kB/s eta 0:01:30
   -------------------- ------------------- 8.0/15.8 MB 87.1 kB/s eta 0:01:30
   -------------------- ------------------- 8.0/15.8 MB 87.1 kB/s eta 0:01:30
   -------------------- ------------------- 8.0/15.8 MB 87.1 kB/s eta 0:01:30
   -------------------- ------------------- 8.0/15.8 MB 87.3 kB/s eta 0:01:29
   -------------------- ------------------- 8.0/15.8 MB 87.3 kB/s eta 0:01:29
   -------------------- ------------------- 8.0/15.8 MB 87.3 kB/s eta 0:01:29
   -------------------- ------------------- 8.0/15.8 MB 87.3 kB/s eta 0:01:29
   -------------------- ------------------- 8.0/15.8 MB 87.3 kB/